Cluster images based on pixel rgb arrays

In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
# from pyspark.mllib.feature import HashingTF, IDF
# from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import KMeans, KMeansModel
from StringIO import StringIO
from PIL import Image
import numpy as np
import csv
import os, tempfile
import boto
import datetime

In [1]:
# AWS S3 credentials:

AWS_KEY = ""
AWS_SECRET = ""
sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", AWS_KEY)
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", AWS_SECRET)

In [3]:
directory = 's3n://amlyelp/subset/trainnew/'

In [11]:
images = sc.binaryFiles(directory)

In [14]:
# convert image files to arrays of pixel RGB
image_to_array = lambda rawdata: np.asarray(Image.open(StringIO(rawdata)))
image_array = images.map(lambda x: (x[0],image_to_array(x[1])))
image_array_flatten = image_array.map(lambda x: (x[0],x[1].flatten())).cache()
del image_array
del images

In [12]:
# train Kmeans cluster on image arrays
clusters = KMeans.train(image_array_flatten.values(), 50, maxIterations=100)

In [31]:
# save Kmeans model
clusters.save(sc, 's3n://amlyelp/subset/model/kmeans/'+str(datetime.datetime.now()).replace(' ', '_')+'/')
# clusters = KMeansModel.load(sc, 'model/test_kmeans')

In [32]:
# predict the image clusters
image_cluster = image_array_flatten.map(lambda x: (x[0].split('/')[-1].replace('.jpg', ''),clusters.predict(x[1])))

In [33]:
image_cluster_list = image_cluster.collect()

In [46]:
# write clusters to temp file
tmpf = tempfile.TemporaryFile()
writer = csv.writer(tmpf)
writer.writerows(image_cluster_list)
tmpf.seek(0)

In [44]:
from boto.s3.connection import S3Connection
from boto.s3.key import Key

AWS_KEY = ""
AWS_SECRET = ""

conn = S3Connection(AWS_KEY, AWS_SECRET, host='s3.amazonaws.com')
pb = conn.get_bucket('amlyelp')

k = Key(pb)

In [47]:
# write cluster predictions to S3
s3_file = "image_cluster_%s.csv" % str(datetime.datetime.now()).replace(' ', '_').replace('.','_')
s3_path = "subset/image_cluster_result"
file_name_to_use_in_s3 = "%s/%s"%(s3_path, s3_file)
k.name = file_name_to_use_in_s3
k.set_contents_from_file(tmpf)

2116